In [3]:
from selenium import webdriver
import requests
from bs4 import BeautifulSoup as bs

import pandas as pd
import time

In [29]:
def get_news(daum_url_list,like_fix,daum_num):

    title_list = []       # 제목
    news_date_list = []   # 날짜
    contents_list = []    # 댓글
    like_num_list = []    # 좋아요 수
    
    driver = webdriver.Chrome("./chromedriver.exe")

    for i in range(daum_num):
        
        driver.get(daum_url_list[i])  # url진입
        print(daum_url_list[i])       
        time.sleep(3)

        try:                            # 더보기 항목이 없을 때까지 클릭
            while driver.find_element_by_xpath('/html/body/div[2]/div[3]/div[2]/div[1]/div[2]/div[6]/div/div/div/div/div[3]/div[2]/button').text !='': 
                driver.find_element_by_xpath('/html/body/div[2]/div[3]/div[2]/div[1]/div[2]/div[6]/div/div/div/div/div[3]/div[2]/button').click()         
                time.sleep(3)
        except: pass

        time.sleep(2)
        
        # url 당 10개의 댓글 확인
        for check in range(1,10):
            try:             
                # 좋아요 수 체크
                like_num = driver.find_element_by_xpath("/html/body/div[2]/div[3]/div[2]/div[1]/div[2]/div[6]/div/div/div/div/div[3]/ul[2]/li[" +str(check)+ "]/div/div/span[2]/button[1]/span[2]").text

                # 좋아요 수가 정해진 값을 넘으면 출력
                if int(like_num) >= like_fix:
                    
                    title = driver.find_element_by_xpath("/html/body/div[2]/div[3]/div[1]/div/h3").text # 제목 추출

                    news_date = driver.find_element_by_css_selector('.num_date')  # 날짜 추출
                    news_date = news_date.text[:11]  
                    
                    contents = driver.find_element_by_xpath('/html/body/div[2]/div[3]/div[2]/div[1]/div[2]/div[6]/div/div/div/div/div[3]/ul[2]/li[' +str(check)+']/div/p').text # 댓글 추출
                    contents = contents.replace('\n'," ")  # 줄 바꿈 무시
                   
                    news_date_list.append(news_date)
                    title_list.append(title)
                    contents_list.append(contents)
                    like_num_list.append(like_num)
            except: pass

        # dateframe 형식으로 만들어줌
        df = pd.DataFrame({'Date':news_date_list,\
                'Title':title_list,\
                'Content':contents_list,\
                'Like':like_num_list})
        
        df.to_csv('daum_crawling_long.csv') 

In [ ]:
def crawling(query,daum_num,like_fix):

    page = 1                                    # 페이지 넘기기
    daum_url_list = []
    while len(daum_url_list) < daum_num:        # 사이트 수를 맞출 때까지
        url = 'https://search.daum.net/search?w=news&sort=recency&q='+query+'&cluster=n&DA=STC&dc=STC&pg=1&r=1&p='+str(page)+'&rc=1&at=more'
        web = requests.get(url).content
        source = bs(web,'html.parser')

        for urls in source.find_all('a',{'class':'f_link_b'}):  
            if urls['href'][-3:] == 'f=o':                                       # 다음뉴스 url 확인
                upload_time = urls['href'][-21:-4]                               # 날짜만 가져옴
                daum_url = "http://news.v.daum.net/v/" + upload_time             # 다음 뉴스 url 만들어줌
                daum_url_list.append(daum_url)
        page += 1                                                                # 사이트 수를 맞출때까지 페이즈를 넘겨라
    get_news(daum_url_list,like_fix,daum_num)

In [35]:
 def crawling(query):
        
    url = 'https://search.daum.net/search?w=news&sort=recency&q='+query+'&cluster=n&DA=STC&dc=STC&pg=1&r=1&p=1&rc=1&at=more&sd=&ed=&period='
    web = requests.get(url).content
    source = bs(web,"html.parser")
    f = open('daum_crawling.csv','w',encoding='utf-8')

    date_list = []            # 날짜
    all_url_content = []      # 페이지에 있는 모든 뉴스 url 리스트
    daum_url_list = []        # 다음뉴스의 url의 리스트
    url_num = 10              # 추출할 url 수
    page = 1                  # 페이지 넘기기


    while len(daum_url_list) < url_num:
        print(page)
        page += 1
        for urls in source.find_all('a',{'class' : 'f_link_b'}):
            all_url_content.append(urls['href']) 
        for i in all_url_content:                                          # 무조건 다음뉴스의 url은 f=o으로 끝남
            if i[-3:] == "f=o":
                upload_time = i[-21:-4]                                     # 기사 업로드 시간만 슬라이싱

    #             date = upload_time[:8]                                      # 업로드날짜
    #             date_list.append(date)

                daum_url = "http://news.v.daum.net/v/" + upload_time  # daum 뉴스 url 형식에 기사 업로드 시간 융합
                daum_url_list.append(daum_url)
                print(daum_url)
#         driver.find_element_by_xpath('/html/body/article/article/div/article/div/div[2]/div[1]/div[4]/span/span[2]/a['+str(page)+']').click()

    # print(urls_list)

In [36]:
if __name__=='__main__':
    query = '공공의대'
    like_fix = 10   # 좋아요 수 
    daum_num = 100  # 사이트 수
    crawling(query,daum_num,like_fix)

1
http://news.v.daum.net/v/20200921112614532
http://news.v.daum.net/v/20200921110328322
http://news.v.daum.net/v/20200921091037348
2
http://news.v.daum.net/v/20200921112614532
http://news.v.daum.net/v/20200921110328322
http://news.v.daum.net/v/20200921091037348
http://news.v.daum.net/v/20200921112614532
http://news.v.daum.net/v/20200921110328322
http://news.v.daum.net/v/20200921091037348
3
http://news.v.daum.net/v/20200921112614532
http://news.v.daum.net/v/20200921110328322
http://news.v.daum.net/v/20200921091037348
http://news.v.daum.net/v/20200921112614532
http://news.v.daum.net/v/20200921110328322
http://news.v.daum.net/v/20200921091037348
http://news.v.daum.net/v/20200921112614532
http://news.v.daum.net/v/20200921110328322
http://news.v.daum.net/v/20200921091037348
